In [1]:
#######################################################################
# Copyright (C)                                                       #
# 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)                  #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################
#######################################################################
# Copyright (C)                                                       #
# 2017 Cheung Auyeung(cheung.auyeung@gmail.com)                       #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

# Chapter 3 Finite Markov Decision Processes

In [2]:
from __future__ import print_function
import numpy as np
import time 

A python decorator to time the elapse time of a function call.

In [3]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r  %2.2f s' % \
            (method.__name__, (te - ts) ))
        return result

    return timed

## Grid World Example
```
 +---+---+---+---+---+
 |   | A |   | B |   |
 +---+---+---+---+---+
 |   |   |   |   |   |
 +---+---+---+---+---+
 |   |   |   | B'|   |
 +---+---+---+---+---+
 |   |   |   |   |   |
 +---+---+---+---+---+
 |   | A'|   |   |   |
 +---+---+---+---+---+ 
```

Above is a 5x5 rectangular gridworld representation of a simple finite MDP. 
The cells of the grid correspond to the states of the **environment**. It has designated location A, A', B and B' for specific behavior of the enviroment to be explained later.

In [4]:
WORLD_SIZE = 5

A_POS = [0, 1]
A_PRIME_POS = [4, 1]
B_POS = [0, 3]
B_PRIME_POS = [2, 3]

Possible actions (decision) of the agent at it's current cell (state):
* Left, Up, Right, Down

In [5]:
# left, up, right, down
actions = ['L', 'U', 'R', 'D']

The action is the decision make by the agent to move in the respective direction to the next cell. Because of the nature of the environment, the agent may or or may not be able to move in the decided direction. 

The enviroment specifies the next cell and the next reward as a result of the action taken by the agent at a cell according to the followings: 
* If the action would take the agent off the grid, the agent shall stay in the same cell and the agent shall receive a reward of -1.
* Else if the agent is at A, the agent shall jump to A' and recive a reward of 10, for all action.
* Else if the agent is at B, the agent shall jump to B' and recive a reward of 5, for all action.
* Else, the agent moves by one cell in the direction of the action and receive a reward of 0.

**Note** In this example, the next state $s'(s,a)$ (i.e. `nextSate`) and reward $r(s,a)$ (i.e, `actionReward`) are deterministic function of the current state and action.

In [6]:
nextState = []
actionReward = []
for i in range(0, WORLD_SIZE):
    nextState.append([])
    actionReward.append([])
    for j in range(0, WORLD_SIZE):
        next = dict()
        reward = dict()
        if i == 0:
            next['U'] = [i, j]
            reward['U'] = -1.0
        else:
            next['U'] = [i - 1, j]
            reward['U'] = 0.0

        if i == WORLD_SIZE - 1:
            next['D'] = [i, j]
            reward['D'] = -1.0
        else:
            next['D'] = [i + 1, j]
            reward['D'] = 0.0

        if j == 0:
            next['L'] = [i, j]
            reward['L'] = -1.0
        else:
            next['L'] = [i, j - 1]
            reward['L'] = 0.0

        if j == WORLD_SIZE - 1:
            next['R'] = [i, j]
            reward['R'] = -1.0
        else:
            next['R'] = [i, j + 1]
            reward['R'] = 0.0

        if [i, j] == A_POS:
            next['L'] = next['R'] = next['D'] = next['U'] = A_PRIME_POS
            reward['L'] = reward['R'] = reward['D'] = reward['U'] = 10.0

        if [i, j] == B_POS:
            next['L'] = next['R'] = next['D'] = next['U'] = B_PRIME_POS
            reward['L'] = reward['R'] = reward['D'] = reward['U'] = 5.0

        nextState[i].append(next)
        actionReward[i].append(reward)

# Figure 3.5  Grid Word Example with Equiprobable Random Policy

The policy of an agent is the  probability $\pi(a\,|\,s)$ that the agent shall take action $a$ at state $s$.  For Figure 3.5, $\pi(a\,|\,s)$ is `actionProb`, which is equiprobable in all directions (action) and at all cell locations (state).

In [7]:
actionProb = []
for i in range(0, WORLD_SIZE):
    actionProb.append([])
    for j in range(0, WORLD_SIZE):
        actionProb[i].append(dict({'L':0.25, 'U':0.25, 'R':0.25, 'D':0.25}))
        

For this equiprobable random policy $\pi(s\,|\,a)$ and discount rate $\gamma = 0.9$ for the reward, the value function $v_\pi(s)$, `world`, is computed by solving the system of linear equations (3.12) iteratively.

$$
v_\pi(s) = \sum_{a} \pi(a\,|\,s) \sum_{s',r} p(s',r\,|\,s,a) 
\left[ r + \gamma v_\pi(s') \right]
$$

Since the mapping from $(s,a)$ to the next state and reward $(s',r)$ is deterministic, we have

$$
v_\pi(s) = \sum_{a} \pi(a\,|\,s) \left[ r(s,a) + \gamma v_\pi(s'(s,a)) \right]
$$
where the reward $r(s,a)$ and the next state $s'(s,a)$ are deterministic functions of $(s,a)$.

The solution $v_\pi(s)$ for all $s$ is a fixed point of the equation. It is computed iteratively by the following algorithm:
1. At $n=0$, initialize $v_\pi(s_n) = 0$ (the vector `world`) by the following code: 
```python
world = np.zeros((WORLD_SIZE, WORLD_SIZE))
```
2. Given $s_{n}$, update $v_\pi(s_{n+1})$ (the vector `newWorld`),   

  $$
  v_\pi(s_{n+1}) = \sum_{a} \pi(a\,|\,s_n) \left[ r(s_n,a) + \gamma v_\pi(s'(s_n,a)) \right]
  $$
  by the following code:
```python 
newWorld[i, j] += actionProb[i][j][action] * 
    (actionReward[i][j][action] + discount * world[newPosition[0], newPosition[1]])
```
3. Repeat (2) until it is converged, i.e., the L1 norm $||v_\pi(s_{n+1}) - v_\pi(s_{n}||_1$, is sufficient small, by the following code:
```python
np.sum(np.abs(world - newWorld)) < 1e-4
```

In [8]:
def figure3_5() :
    discount = 0.9
    world = np.zeros((WORLD_SIZE, WORLD_SIZE))
    n = 0
    while True:
        # keep iteration until convergence
        newWorld = np.zeros((WORLD_SIZE, WORLD_SIZE))
        for i in range(0, WORLD_SIZE):
            for j in range(0, WORLD_SIZE):
                for action in actions:
                    newPosition = nextState[i][j][action]
                    # bellman equation
                    newWorld[i, j] += actionProb[i][j][action] * (
                        actionReward[i][j][action] + 
                        discount * world[newPosition[0], newPosition[1]] )
        n += 1
        if np.sum(np.abs(world - newWorld)) < 1e-4:
            print(
                'Value Function under the Random Policy found after {} iterations:'.format(n))
            return newWorld
        world = newWorld

In [9]:
state_value = figure3_5()
print(np.array_str(state_value, precision=1))

Value Function under the Random Policy found after 77 iterations:
[[ 3.3  8.8  4.4  5.3  1.5]
 [ 1.5  3.   2.3  1.9  0.5]
 [ 0.1  0.7  0.7  0.4 -0.4]
 [-1.  -0.4 -0.4 -0.6 -1.2]
 [-1.9 -1.3 -1.2 -1.4 -2. ]]


## Figure 3.8 Grid Word Example with Optimal Policy

Figure 3.8 also try to solve the same grid world problem in Figure 3.5 with discount rate $\gamma = 0.9$ for the reward but with optimal policy.

The value function with the optimal policy is found by the
Bellman optimalality equation for $v_\ast(s)$ 

$$
\begin{align}
v_\ast(s) \,=\, &\underset{a}{\mathrm{argmax}}\, q_{\pi_\ast}(s,a) \\
 \,=\, &\underset{a}{\mathrm{argmax}}\, \sum_{s',r} p(s',r\,|\,s,a) \left[ r + \gamma v_\ast(s') \right]
\end{align}
$$

Since the mapping from $(s,a)$ to the next state and reward $(s',r)$ is deterministic, we have

$$
\begin{align}
v_\ast(s) \,=\, &\underset{a}{\mathrm{argmax}}\, q_{\pi_\ast}(s,a) \\
 \,=\, &\underset{a}{\mathrm{argmax}}\, \left[ r(s,a) + \gamma v_\ast(s'(s,a)) \right]
\end{align}
$$
where the reward $r(s,a)$ and the next state $s'(s,a)$ are deterministic functions of $(s,a)$.

The solution $v_\ast(s)$ for all $s$ is a fixed point of the equation. It is computed iteratively by the following algorithm:
1. At $n=0$, initialize $v_\ast(s_n) = 0$ (the vector `world`) by the following code: 
    ```python
    world = np.zeros((WORLD_SIZE, WORLD_SIZE))
    ```
2. for every $s_{n}$ (i.e. every grid position `[i][j]` in the $n$-th iteration, compute the `values`  :    

  $$
     q_{\pi_\ast}(s_n,a) = r(s,a) + \gamma v_\ast(s'(s_n,a))
  $$ 
   
   for an action $a$ (i.e. `[action]`) by the following code :
```pyhon 
values.append(
    actionReward[i][j][action] + discount * world[newPosition[0], newPosition[1]])
```
Then update $v_\ast(s_{n+1})$ for interation $n+1$, (the vector `newWorld`), 
    
    $$
    v_\ast(s_{n+1}) = \underset{a}{\mathrm{argmax}}\, q_{\pi_\ast}(s_n,a)
    $$
    by the following code:
```python           
newWorld[i][j] = np.max(values)
```
3. Repeat (2) until it is converged, i.e., the L1 norm $||v_\ast(s_{n+1}) - v_\ast(s_{n}||_1$, is sufficient small, by the following code:
```python
np.sum(np.abs(world - newWorld)) < 1e-4
```    

In [10]:
def figure3_8() :
    discount = 0.9
    world = np.zeros((WORLD_SIZE, WORLD_SIZE))
    n = 0
    while True:
        # keep iteration until convergence
        newWorld = np.zeros((WORLD_SIZE, WORLD_SIZE))
        for i in range(0, WORLD_SIZE):
            for j in range(0, WORLD_SIZE):
                values = []
                for action in actions:
                    newPosition = nextState[i][j][action]
                    # value iteration
                    values.append(
                        actionReward[i][j][action] + 
                        discount * world[newPosition[0], newPosition[1]])
                newWorld[i][j] = np.max(values)
        n += 1
        if np.sum(np.abs(world - newWorld)) < 1e-4:
            print('Optimal Value Function found after {} iterations:'.format(n))            
            return(newWorld)
        world = newWorld

In [11]:
optimal_state_value = figure3_8()
print(np.array_str(optimal_state_value, precision=1))

Optimal Value Function found after 124 iterations:
[[ 22.   24.4  22.   19.4  17.5]
 [ 19.8  22.   19.8  17.8  16. ]
 [ 17.8  19.8  17.8  16.   14.4]
 [ 16.   17.8  16.   14.4  13. ]
 [ 14.4  16.   14.4  13.   11.7]]
